# Équipe 7
## Notebook de Foulémata

In [1]:
import pandas as pd 
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

from jupyter_dash import JupyterDash
import dash           
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output


In [2]:
def load_genre_data():
    """
    Récupération des catégories pour chaque film
    Retourne: un dataframe avec un seul genre par film
    """
    # Chargement du fichiers contenant le genre pour chaque film
    df = pd.read_csv("../Src/Assets/Data/_film_genres.csv")
    df["genres_categorized"] = df["genres_categorized"].replace("FICTION SPÉCULATIVE","SCIENCE FICTION")
    
    # Suppression des données dupliquées et des valeurs manquantes NaN 
    df.drop_duplicates(subset=['titreOriginal','genres_categorized','anneeSortie'], \
        keep='first', inplace=True) 
    df.drop_duplicates(subset=['titreOriginal','anneeSortie'], keep='first', inplace=True)
    df.dropna(inplace=True)

    #Conversion de la colonne anneeSortie (from float to int)
    df.anneeSortie = df.anneeSortie.astype(int)

    #Correction pour le film "Dark Feed" (2012, non 2201)
    df["anneeSortie"] = df["anneeSortie"].replace(2201,2012)
    df_movies = df

    return df_movies


In [3]:
def bumpChart_preprocess(df_movies):
    """
        Decrire la fonction
    """
    #Classer les films selon l'année de production par ordre croissant
    df_movies = df_movies.sort_values(by="anneeSortie")

    #Affiché le nombre de film produit selon le genre tous les 10 ans - Création d'une colonne 'year' contenant les années de 1900 à 2019 avec un interval de 10 ans.
    annee_x = [*range(1900, 2019, 10)]
    for i in annee_x:
        df_movies.loc[(df_movies['anneeSortie'] > i) & (df_movies['anneeSortie'] <= (i+10)), 'year'] = i+10
    df_movies.loc[(df_movies['anneeSortie'] <= 1900), 'year'] = 1900
    df_movies.loc[(df_movies['anneeSortie'] > 2010), 'year'] = 2019
    ###Pas assez de genre différents avant 1900 (non inclus dans le graph final)
    df_movies = df_movies[(df_movies["year"] != 1900)] 
    df_movies.year = df_movies.year.astype(int)

    #Regrouper les données par année et par genre afin de compter le nombre de film par année selon le genre
    df_10yearsGap = df_movies.groupby(['year','genres_categorized']).size().reset_index(name="counts")

    return df_10yearsGap


In [4]:
def get_genre_ranked(df_10yearsGap):
    """
    Obtenir le classement des genres tous les 10 ans. Création d'une colonne "rank" avec le rang du genre 
    """
    #Sort within group
    df_10yearsGap = df_10yearsGap.groupby(['year']) \
        .apply(lambda x: x.sort_values(["counts"], ascending = False)) \
            .reset_index(drop=True)
    df_10yearsGap["rank"] = df_10yearsGap.groupby('year')["counts"] \
        .rank(method="first", ascending = False)
    
    #Conversion de la colonne rank (from float to int)
    df_10yearsGap["rank"] = df_10yearsGap["rank"].astype(int)

    df_genre_ranked = df_10yearsGap

    return df_genre_ranked

In [5]:
def draw_bumpChart(df):
    '''
        Draws the bumpChart
    '''
    fig = go.Figure()

    #Tracé pour chaque genre
    x_data = df["year"]
    y_data = df["rank"]
    counts = df.counts
    genreCat = df.genres_categorized
    
    temporary_theme=px.colors.sequential.dense[2:] + list(reversed(px.colors.sequential.RdPu))[:-2]
    all_genre = df.genres_categorized.unique()
    
    #Tracé le graphe en fonction du genre selectionné 
    for genre,i in zip(all_genre,temporary_theme):
        fig.add_trace(go.Scatter(
            x = x_data[genreCat == str(genre)],
            y = y_data[genreCat == str(genre)],
            name=genre,
            mode='lines+markers',
            marker_color= i,
            yaxis="y",
        )),
    
    #Carré d'arrière-plan
    x_bg = np.repeat([*range(1910, 2019, 10), 2019],max(y_data))
    y_bg = [*range(1,max(y_data)+1,1)]*max(y_data)
    fig.add_trace(go.Scatter(
        x=x_bg,
        y=y_bg,
        mode='markers',
        marker_symbol='square',
        marker_color="lightgrey",
        yaxis="y2",
        ))
    

    return fig

In [6]:
def update_axis(fig, df):
    '''
        Updates the x axis and the two y axes 
        Args:fig, df
        Returns: The updated figure
    '''

    x_data = df["year"]
    y_data = df["rank"]
    y_genre = df["genres_categorized"]
    rank_range_descending = [max(df["rank"])+1, 0]
    rank_range_ascending = [*range(min(y_data),max(y_data)+1,1)]

    fig = go.Figure(fig)
        
    fig.update_layout(
            xaxis=dict(
                title="Année de production",
                tickmode = 'array',
                tickvals= x_data,
                ticktext= x_data,
                tickangle = -45,
                fixedrange=True,
                scaleratio = 4,
            ),
            yaxis=dict(
                range=rank_range_descending,
                tickmode="array",
                tickvals= y_data[x_data == max(x_data)],
                ticktext= y_genre[x_data == max(x_data)],
                showgrid= False,
                anchor="x",
                overlaying="y2",
                side="right",
                scaleratio = 4,
                #position=0.15
                ),
            yaxis2=dict(
                title="Rang",
                range=rank_range_descending,
                tickmode="array",
                tickvals= rank_range_ascending,
                ticktext= rank_range_ascending,
                anchor="x",
                side="left",
                constrain='domain',
                #position=0.15
                ),
            showlegend=False,
            paper_bgcolor='rgba(0,0,0,0)',
            plot_bgcolor='rgba(0,0,0,0)'
            )

    return fig

In [7]:
def prep_data():
    df_movies = load_genre_data()
    df_10yearsGap = bumpChart_preprocess(df_movies)
    df = get_genre_ranked(df_10yearsGap)
    return df

def init_bumpChart():
    df = prep_data()
    fig = draw_bumpChart(df)
    fig = update_axis(fig, df)
    return fig

def update_bumpChart(df, genre_chosen):
    fig = init_bumpChart()
    fig = fig.update_traces(marker_color="lightgrey")
    x_data = df["year"]
    y_data = df["rank"]
    genreCat = df.genres_categorized
    fig.add_trace(go.Scatter(
        x = x_data[genreCat == str(genre_chosen)],
        y = y_data[genreCat == str(genre_chosen)],
        name=genre_chosen,
        mode='lines+markers',
        marker_symbol='square',
        marker_color='red',
        marker_size=10,
        hovertemplate=None,
    )),
    return fig

In [9]:
external_stylesheets=['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

df = prep_data()
all_genre = df.genres_categorized.unique()


app.layout = html.Div(className='content', children=[
    html.Header(children=[
        html.H3(['Evolution des genres au cours du temps'],
                style={'font-weight': 'bold', "text-align": "center"}),
    ]),
    html.Main(className='bumpChart-container', children=[
        html.Label(['Choisis un genre pour suivre son évolution dans le temps: '],
            style={'font-weight': 'bold', "text-align": "left"}),
        dcc.Dropdown(
            id='bumpChart_dropdown',
            options=[
                {'label': i, 'value': j} 
                for i,j in zip (np.append(all_genre,"Tous les genres"), np.append(all_genre,"all_chosen"))],
            value="all_chosen",
            placeholder='Selectionne un genre...',
            multi=False,
            clearable=True,
            style={'width':"50%"},
        ),
        dcc.Graph(
            id='bumpChart',
            className='graph',
            figure = init_bumpChart(),
            config=dict(
                scrollZoom=False,
                showTips=False,
                showAxisDragHandles=False,
                doubleClick=False,
                displayModeBar=False
            )
        ),
        
    ])
])

# On connecte le Dropdown avec le bump chart
@app.callback(
    Output(component_id='bumpChart', component_property='figure'),
    [Input(component_id='bumpChart_dropdown', component_property='value')],
)

def update_graph(genre_chosen):
    if genre_chosen != "all_chosen":
        fig = update_bumpChart(df, genre_chosen)
    else:
        fig = init_bumpChart()
    return fig

app.run_server(mode='external', port = 8020, dev_tools_ui=True, #debug=True, 
               dev_tools_hot_reload =True, threaded=True)

Dash app running on http://127.0.0.1:8020/
